# Sistema ALPR - Ejemplo Completo

Este notebook demuestra cómo usar el sistema ALPR completo para:
1. Detectar placas en imágenes
2. Segmentar caracteres
3. Reconocer texto con CNN
4. Visualizar resultados

In [ ]:
# Imports
import sys
sys.path.append('..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from config import create_directories, CLASSIFIER_MODEL_DIR, CROPS_DIR
from pipeline.alpr_pipeline import ALPRPipeline
from detectors.plate_detector import PlateDetector
from segmenters.classical_segmenter import ClassicalSegmenter

%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)

## 1. Configuración

In [ ]:
# Crear directorios necesarios
create_directories()

# Configurar rutas
CLASSIFIER_PATH = Path(CLASSIFIER_MODEL_DIR) / "classifier.h5"

print(f"Buscando clasificador en: {CLASSIFIER_PATH}")
print(f"Existe: {CLASSIFIER_PATH.exists()}")

## 2. Inicializar Pipeline

In [ ]:
# Inicializar pipeline completo
pipeline = ALPRPipeline(
    classifier_model=str(CLASSIFIER_PATH) if CLASSIFIER_PATH.exists() else None,
    use_easyocr=True,
    segmenter_type="classical"
)

print("✓ Pipeline inicializado")

## 3. Procesar una Imagen

In [ ]:
# Ruta a tu imagen de prueba
IMAGE_PATH = "../examples/test_image.jpg"  # Cambia esto a tu imagen

# Verificar que existe
if not Path(IMAGE_PATH).exists():
    print(f"⚠ Advertencia: {IMAGE_PATH} no existe")
    print("Por favor, coloca una imagen de prueba en la carpeta examples/")
else:
    # Cargar y mostrar imagen original
    img = cv2.imread(IMAGE_PATH)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(img_rgb)
    plt.title("Imagen Original")
    plt.axis('off')
    plt.show()

In [ ]:
# Procesar imagen
if Path(IMAGE_PATH).exists():
    results = pipeline.recognize_from_path(
        IMAGE_PATH,
        save_crops=True,
        visualize=True
    )
    
    print("\n" + "="*60)
    print("RESULTADOS")
    print("="*60)
    
    for idx, result in enumerate(results):
        print(f"\nPlaca {idx + 1}:")
        print(f"  Texto:      {result['plate']}")
        print(f"  Método:     {result['method']}")
        print(f"  Confianza:  {result['conf']:.2%}")

## 4. Visualizar Detecciones

In [ ]:
# Detectar placas y visualizar
if Path(IMAGE_PATH).exists():
    detector = PlateDetector()
    img = cv2.imread(IMAGE_PATH)
    plates = detector.detect(img)
    
    # Dibujar bounding boxes
    img_vis = img.copy()
    for plate_data in plates:
        x1, y1, x2, y2 = plate_data['bbox']
        conf = plate_data['conf']
        
        cv2.rectangle(img_vis, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(img_vis, f"{conf:.2f}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB))
    plt.title(f"Detecciones: {len(plates)} placa(s)")
    plt.axis('off')
    plt.show()

## 5. Visualizar Segmentación de Caracteres

In [ ]:
# Segmentar y visualizar caracteres
if Path(IMAGE_PATH).exists() and len(plates) > 0:
    segmenter = ClassicalSegmenter()
    
    for plate_idx, plate_data in enumerate(plates):
        plate_img = plate_data['crop']
        chars = segmenter.segment(plate_img)
        
        print(f"\nPlaca {plate_idx + 1}: {len(chars)} caracteres detectados")
        
        # Mostrar placa
        plt.figure(figsize=(12, 3))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB))
        plt.title("Placa Detectada")
        plt.axis('off')
        
        # Mostrar caracteres
        if chars:
            plt.subplot(1, 2, 2)
            n_chars = len(chars)
            combined = np.hstack([cv2.resize(ch, (50, 80)) for ch in chars[:8]])  # max 8
            plt.imshow(combined, cmap='gray')
            plt.title(f"Caracteres Segmentados ({n_chars})")
            plt.axis('off')
        
        plt.tight_layout()
        plt.show()

## 6. Procesar Múltiples Imágenes

In [ ]:
# Procesar múltiples imágenes desde un directorio
IMAGES_DIR = Path("../examples")

if IMAGES_DIR.exists():
    image_files = list(IMAGES_DIR.glob("*.jpg")) + list(IMAGES_DIR.glob("*.png"))
    
    print(f"Encontradas {len(image_files)} imágenes\n")
    
    all_results = []
    
    for img_path in image_files[:5]:  # Procesar máximo 5
        print(f"Procesando: {img_path.name}")
        try:
            results = pipeline.recognize_from_path(img_path, save_crops=False, visualize=False)
            for result in results:
                all_results.append({
                    'image': img_path.name,
                    'plate': result['plate'],
                    'confidence': result['conf'],
                    'method': result['method']
                })
                print(f"  → {result['plate']} (conf: {result['conf']:.2f})")
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    # Mostrar resultados en tabla
    if all_results:
        import pandas as pd
        df = pd.DataFrame(all_results)
        print("\n" + "="*60)
        print(df.to_string(index=False))
else:
    print(f"Directorio {IMAGES_DIR} no existe")

## 7. Estadísticas y Análisis

In [ ]:
# Análisis de resultados
if all_results:
    import pandas as pd
    df = pd.DataFrame(all_results)
    
    print("ESTADÍSTICAS")
    print("="*60)
    print(f"Total procesadas:         {len(df)}")
    print(f"Confianza promedio:       {df['confidence'].mean():.2%}")
    print(f"Confianza mínima:         {df['confidence'].min():.2%}")
    print(f"Confianza máxima:         {df['confidence'].max():.2%}")
    print("\nDistribución de métodos:")
    print(df['method'].value_counts())
    
    # Gráfico de confianzas
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.hist(df['confidence'], bins=10, edgecolor='black')
    plt.xlabel('Confianza')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de Confianzas')
    
    plt.subplot(1, 2, 2)
    df['method'].value_counts().plot(kind='bar')
    plt.xlabel('Método')
    plt.ylabel('Cantidad')
    plt.title('Métodos Utilizados')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

## 8. Probar Componentes Individuales

In [ ]:
# Probar solo detector
print("Probando PlateDetector...")
detector = PlateDetector()
if Path(IMAGE_PATH).exists():
    img = cv2.imread(IMAGE_PATH)
    plates = detector.detect(img, conf=0.3)
    print(f"✓ Detectadas {len(plates)} placa(s)")
    for i, p in enumerate(plates):
        print(f"  Placa {i+1}: bbox={p['bbox']}, conf={p['conf']:.3f}")

In [ ]:
# Probar solo segmentador
print("Probando ClassicalSegmenter...")
segmenter = ClassicalSegmenter()
if plates:
    plate_img = plates[0]['crop']
    chars = segmenter.segment(plate_img)
    print(f"✓ Segmentados {len(chars)} caractere(s)")

## 9. Guardar Resultados

In [ ]:
# Guardar resultados en CSV
if all_results:
    output_file = "../results/notebook_results.csv"
    df.to_csv(output_file, index=False)
    print(f"✓ Resultados guardados en: {output_file}")

## Conclusión

Este notebook demostró:
- ✓ Inicialización del pipeline ALPR
- ✓ Detección de placas con YOLOv8
- ✓ Segmentación de caracteres
- ✓ Reconocimiento con CNN
- ✓ Visualización de resultados
- ✓ Análisis estadístico

### Próximos pasos:
1. Entrenar modelos con tus propios datos
2. Ajustar parámetros en `config.py`
3. Evaluar con `evaluate.py`
4. Optimizar para producción